# iperf3 experiments — logbook

In [ ]:
## Commands used
#experiment 1:
%%bash
set -euo pipefail
RUN=$(date +%Y%m%d-%H%M%S)

# i) start local server (separate terminal in real runs; kept for documentation)
# iperf3 -s

# ii) TCP (10 s) to localhost
iperf3 -c 127.0.0.1 -t 10 -J > RawData/exp1_tcp_localhost_${RUN}.json

# 100 Mb/s
iperf3 -u -c 127.0.0.1 -b 100M -t 10 -J > RawData/exp1_udp_100M_${RUN}.json

# 1000 Mb/s (1 Gb/s)
iperf3 -u -c 127.0.0.1 -b 1000M -t 10 -J > RawData/exp1_udp_1000M_${RUN}.json

# 10 Gb/s
iperf3 -u -c 127.0.0.1 -b 10G -t 10 -J > RawData/exp1_udp_10G_${RUN}.json

#experiment 2:
%%bash
set -euo pipefail
RUN=$(date +%Y%m%d-%H%M%S)
HOST=iperf3.moji.fr
PORT=5202

# sanity TCP reachability (short)
iperf3 -4 -c $HOST -p $PORT -t 1

# UDP 10 Mb/s and 100 Mb/s
iperf3 -u -4 -c $HOST -p $PORT -b 10M  -t 10 -J > RawData/exp2_udp_10M_${HOST}_${PORT}_${RUN}.json
iperf3 -u -4 -c $HOST -p $PORT -b 100M -t 10 -J > RawData/exp2_udp_100M_${HOST}_${PORT}_${RUN}.json


In [ ]:
import sys, subprocess
print(sys.executable)
from pathlib import Path
import json, pandas as pd, re

RAW_DIR = Path("RawData")

def _read_json(p: Path):
    try:
        return json.loads(p.read_text())
    except Exception as e:
        return {"error": f"json_error: {e}"}

def _infer_protocol(d):
    end = d.get("end", {})
    sr = end.get("sum_received") or {}
    ss = end.get("sum_sent") or {}
    if "jitter_ms" in sr or "jitter_ms" in ss or d.get("start", {}).get("target_bitrate"):
        return "UDP"
    return "TCP"

def _name_parts(name: str):
    # exp, host, port from your naming convention (fallbacks if not matched)
    m = re.search(r"(exp\d+)_(tcp|udp).*?_(?P<host>[a-z0-9\.\-]+)_(?P<port>\d+)", name, re.I)
    return {
        "experiment": (m.group(1) if m else None),
        "host": (m.group("host") if m else None),
        "port": int(m.group("port")) if (m and m.group("port")) else None,
    }

def summarize_file(p: Path):
    d = _read_json(p)
    start = d.get("start", {})
    end = d.get("end", {})
    ss = end.get("sum_sent") or {}
    sr = end.get("sum_received") or {}
    # fallback “sum” if neither present
    s_any = end.get("sum") or {}

    parts = _name_parts(p.name)
    proto = _infer_protocol(d)

    # throughput (prefer explicit sent/received if present)
    bps_sender = ss.get("bits_per_second", s_any.get("bits_per_second"))
    bps_receiver = sr.get("bits_per_second", s_any.get("bits_per_second"))

    # reverse detection (heuristic: client requested reverse or receiver side populated while sender missing)
    reverse_flag = bool(d.get("start", {}).get("reverse")) or (bps_receiver and not bps_sender)

    row = {
        "file": p.name,
        **parts,
        "protocol": proto,
        "reverse": reverse_flag,
        "timestamp_utc": start.get("timestamp", {}).get("time"),
        "version": start.get("version"),
        "system_info": start.get("system_info"),
        "duration_s": (end.get("sum", {}) or ss or sr or {}).get("seconds"),
        "bps_sender": bps_sender,
        "bps_receiver": bps_receiver,
        # TCP:
        "retransmits": ss.get("retransmits"),
        # UDP:
        "jitter_ms": (sr.get("jitter_ms") or ss.get("jitter_ms")),
        "lost_percent": (sr.get("lost_percent") or ss.get("lost_percent")),
        "target_bitrate": start.get("target_bitrate"),
        # Surface run-time error if present:
        "error": d.get("error"),
    }
    return row

files = sorted(RAW_DIR.glob("*.json"))
df = pd.DataFrame(summarize_file(p) for p in files)

# Nice-to-haves:
# - Mbps for quick reading
if not df.empty:
    for col in ["bps_sender", "bps_receiver", "target_bitrate"]:
        if col in df:
            df[col.replace("bps", "Mbps")] = df[col] / 1e6

# Save a tidy snapshot for your report/notebook:
# df.to_csv("RawData/_summary.csv", index=False)

df


/home/aces/projects/Networks/networks-proyect-2/.venv/bin/python


,file,experiment,host,port,protocol,reverse,timestamp_utc,version,system_info,duration_s,bps_sender,bps_receiver,retransmits,jitter_ms,lost_percent,target_bitrate,error,Mbps_sender,Mbps_receiver
0,exp1_tcp_localhost_20251016-210340.json,exp1,localhost,20251016,TCP,False,"Fri, 17 Oct 2025 01:05:21 GMT",iperf 3.16,Linux AlexDesktop 5.15.167.4-microsoft-standar...,10.001245,8.252513e+10,8.252464e+10,2.0,NaN,NaN,0.0,None,82525.134875,82524.637365
1,exp1_tcp_localhost_srvout_20251016-210340.json,exp1,srvout,20251016,TCP,False,"Fri, 17 Oct 2025 01:06:28 GMT",iperf 3.16,Linux AlexDesktop 5.15.167.4-microsoft-standar...,10.001185,8.150412e+10,8.150372e+10,1.0,NaN,NaN,0.0,None,81504.123419,81503.723001
2,exp1_udp_1000M_20251016-210702.json,exp1,1000M,20251016,UDP,False,"Fri, 17 Oct 2025 01:09:57 GMT",iperf 3.16,Linux AlexDesktop 5.15.167.4-microsoft-standar...,10.000275,9.998999e+08,9.998946e+08,NaN,0.012190,0.000000,1000.0,None,999.899876,999.894576
3,exp1_udp_100M_20251016-210702.json,exp1,100M,20251016,UDP,False,"Fri, 17 Oct 2025 01:07:08 GMT",iperf 3.16,Linux AlexDesktop 5.15.167.4-microsoft-standar...,10.000283,1.000058e+08,1.000051e+08,NaN,0.003173,0.000000,100.0,None,100.005836,100.005106
4,exp1_udp_10G_20251016-210702.json,exp1,10G,20251016,UDP,False,"Fri, 17 Oct 2025 01:09:23 GMT",iperf 3.16,Linux AlexDesktop 5.15.167.4-microsoft-standar...,10.001717,9.997557e+09,9.809789e+09,NaN,0.001422,1.877622,10000.0,None,9997.557142,9809.788862
5,exp2_tcp_moji_5202_20251016-213110.json,exp2,moji,5202,TCP,False,"Fri, 17 Oct 2025 01:31:20 GMT",iperf 3.16,Linux AlexDesktop 5.15.167.4-microsoft-standar...,10.002082,1.175209e+08,1.058430e+08,589.0,NaN,NaN,0.0,None,117.520902,105.842970
6,exp2_tcp_srvout_moji_5201_20251016-213110.json,exp2,moji,5201,TCP,False,"Fri, 17 Oct 2025 01:31:47 GMT",iperf 3.16,Linux AlexDesktop 5.15.167.4-microsoft-standar...,10.001252,9.886834e+07,9.659206e+07,699.0,NaN,NaN,0.0,None,98.868338,96.592062
7,exp2_udp_100M_moji_5202_20251016-212730.json,exp2,moji,5202,UDP,False,"Fri, 17 Oct 2025 01:29:36 GMT",iperf 3.16,Linux AlexDesktop 5.15.167.4-microsoft-standar...,11.044534,9.998987e+07,9.048458e+07,NaN,0.085096,0.000000,100.0,None,99.989866,90.484578
8,exp2_udp_10M_moji_5202_20251016-212730.json,exp2,moji,5202,UDP,False,"Fri, 17 Oct 2025 01:28:42 GMT",iperf 3.16,Linux AlexDesktop 5.15.167.4-microsoft-standar...,10.299613,9.999167e+06,9.708481e+06,NaN,0.717225,0.000000,10.0,None,9.999167,9.708481
